In [1]:
%pip install -qU langgraph langchain langchain-google-genai langchain-tavily python-dotenv aiosqlite langgraph-checkpoint-sqlite

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY') 
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

In [3]:
import operator
from typing import Annotated, List, Any, Dict
from dataclasses import dataclass, field
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, BaseMessage, AnyMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_tavily import TavilySearch
from langgraph.checkpoint.sqlite import SqliteSaver
from typing_extensions import TypedDict
import sqlite3

d:\Videos - Cursos\TIME_LATAM\Alvaro\Videos_2255\MULTIAGENTES_LANGGRAPH\.venv\Lib\site-packages\langchain_tavily\tavily_research.py:97: UserWarning: Field name "output_schema" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]
d:\Videos - Cursos\TIME_LATAM\Alvaro\Videos_2255\MULTIAGENTES_LANGGRAPH\.venv\Lib\site-packages\langchain_tavily\tavily_research.py:97: UserWarning: Field name "stream" in "TavilyResearch" shadows an attribute in parent "BaseTool"
  class TavilyResearch(BaseTool):  # type: ignore[override, override]


In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

conn = sqlite3.connect("checkpoints.db", check_same_thread=False)
memory = SqliteSaver(conn)

In [5]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        
        graph = StateGraph(AgentState)

        graph.add_node("llm", self.call_gemini)

        graph.add_node("action", self.take_action)

        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})

        graph.add_edge("action", "llm")

        graph.set_entry_point("llm")

        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_gemini(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages

        print("Mensajes enviados al modelo:", messages)
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Llamando la Herramienta: {t['name']} con los siguientes argumentos: {t['args']}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Retornando a la LLM tras la acción!")
        return {'messages': results}

In [6]:
# current_tavily_api_key = os.getenv('TAVILY_API_KEY')
if not TAVILY_API_KEY:
    raise ValueError("TAVILY_API_KEY não encontrada. Certifique-se de que está no seu .env e python-dotenv está instalado.")

tool = TavilySearch(max_results=3, tavily_api_key=TAVILY_API_KEY)


prompt_system = """
Eres un asistente de investigación inteligente. Usa el motor de búsqueda (tavily_search_results_json) para buscar información.
Tienes permiso para realizar múltiples llamadas a la herramienta (de forma conjunta o en secuencia).
Busca información únicamente cuando tengas certeza de qué buscar.
Si necesitas más detalles para formular una pregunta de seguimiento, tienes permiso para hacerlo.
Cuando se te solicite comparar información (por ejemplo: cuál es más caliente, más grande, etc.), utiliza la información del historial de la conversación y los resultados de las herramientas.
"""

model = ChatGoogleGenerativeAI(model="gemini-2.5-pro") 

abot = Agent(model, [tool], system=prompt_system, checkpointer=memory)

In [9]:
messages = [HumanMessage(content="Cómo está el clima en Lima - Perú hoy (29 de diciembre de 2025)?")]
thread = {"configurable": {"thread_id": "3"}} 


print("\n--- Pregunta 1: Clima en Lima ---")
for event in abot.graph.stream({"messages": messages}, thread):
    for k, v in event.items():
        if k in ("llm", "action"): 
             print(f"{k}: {v['messages']}")


--- Pregunta 1: Clima en Lima ---
Mensajes enviados al modelo: [SystemMessage(content='\nEres un asistente de investigación inteligente. Usa el motor de búsqueda (tavily_search_results_json) para buscar información.\nTienes permiso para realizar múltiples llamadas a la herramienta (de forma conjunta o en secuencia).\nBusca información únicamente cuando tengas certeza de qué buscar.\nSi necesitas más detalles para formular una pregunta de seguimiento, tienes permiso para hacerlo.\nCuando se te solicite comparar información (por ejemplo: cuál es más caliente, más grande, etc.), utiliza la información del historial de la conversación y los resultados de las herramientas.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Cómo está el clima en Lima - Perú hoy (29 de diciembre de 2025)?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search', 'arguments': '{"query": "clima en Lima Per\\u00fa en di

In [10]:
messages = [HumanMessage(content="Y el clima en Medellín?")]
thread = {"configurable": {"thread_id": "3"}} 

print("\n--- Pregunta 2: Clima en Medellín ---")
for event in abot.graph.stream({"messages": messages}, thread):
    for k, v in event.items():
        if k in ("llm", "action"):
            print(f"{k}: {v['messages']}")


--- Pregunta 2: Clima en Medellín ---
Mensajes enviados al modelo: [SystemMessage(content='\nEres un asistente de investigación inteligente. Usa el motor de búsqueda (tavily_search_results_json) para buscar información.\nTienes permiso para realizar múltiples llamadas a la herramienta (de forma conjunta o en secuencia).\nBusca información únicamente cuando tengas certeza de qué buscar.\nSi necesitas más detalles para formular una pregunta de seguimiento, tienes permiso para hacerlo.\nCuando se te solicite comparar información (por ejemplo: cuál es más caliente, más grande, etc.), utiliza la información del historial de la conversación y los resultados de las herramientas.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Cómo está el clima en Lima - Perú hoy (29 de diciembre de 2025)?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search', 'arguments': '{"query": "clima en Lima Per\\u00fa e

In [11]:
messages = [HumanMessage(content="Cuál ciudad está más caliente?")]
thread = {"configurable": {"thread_id": "3"}} 

print("\n--- Pregunta 3: Comparación ---")
for event in abot.graph.stream({"messages": messages}, thread):
    for k, v in event.items():
        if k in ("llm", "action"):
            print(f"{k}: {v['messages']}")


--- Pregunta 3: Comparación ---
Mensajes enviados al modelo: [SystemMessage(content='\nEres un asistente de investigación inteligente. Usa el motor de búsqueda (tavily_search_results_json) para buscar información.\nTienes permiso para realizar múltiples llamadas a la herramienta (de forma conjunta o en secuencia).\nBusca información únicamente cuando tengas certeza de qué buscar.\nSi necesitas más detalles para formular una pregunta de seguimiento, tienes permiso para hacerlo.\nCuando se te solicite comparar información (por ejemplo: cuál es más caliente, más grande, etc.), utiliza la información del historial de la conversación y los resultados de las herramientas.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Cómo está el clima en Lima - Perú hoy (29 de diciembre de 2025)?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search', 'arguments': '{"query": "clima en Lima Per\\u00fa en dici

In [12]:
messages = [HumanMessage(content="Cuál ciudad está más caliente?")]
thread = {"configurable": {"thread_id": "3"}} 

print("\n--- Pergunta 4: Comparación ---")
for event in abot.graph.stream({"messages": messages}, thread):
    for k, v in event.items():
        if k in ("llm", "action"):
            print(f"{k}: {v['messages']}")


--- Pergunta 4: Comparación ---
Mensajes enviados al modelo: [SystemMessage(content='\nEres un asistente de investigación inteligente. Usa el motor de búsqueda (tavily_search_results_json) para buscar información.\nTienes permiso para realizar múltiples llamadas a la herramienta (de forma conjunta o en secuencia).\nBusca información únicamente cuando tengas certeza de qué buscar.\nSi necesitas más detalles para formular una pregunta de seguimiento, tienes permiso para hacerlo.\nCuando se te solicite comparar información (por ejemplo: cuál es más caliente, más grande, etc.), utiliza la información del historial de la conversación y los resultados de las herramientas.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Cómo está el clima en Lima - Perú hoy (29 de diciembre de 2025)?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search', 'arguments': '{"query": "clima en Lima Per\\u00fa en dici